In [1]:
comments = '1899_comments.zst'
submissions = '1899_submissions.zst'


import pandas as pd
df_comments= pd.read_json(comments,compression=dict(method='zstd',
                  max_window_size=2147483648), lines=True)

import pandas as pd
df_submissions= pd.read_json(submissions,compression=dict(method='zstd',
                  max_window_size=2147483648), lines=True)


In [2]:
# comments
print("Comments DataFrame shape:", df_comments.shape)
print("Comments DataFrame columns:", df_comments.columns)
print("First few rows of Comments DataFrame:")
print(df_comments.head())

# submissions
print("Submissions DataFrame shape:", df_submissions.shape)
print("Submissions DataFrame columns:", df_submissions.columns)
print("First few rows of Submissions DataFrame:")
print(df_submissions.head())

Comments DataFrame shape: (40489, 56)
Comments DataFrame columns: Index(['all_awardings', 'associated_award', 'author', 'author_created_utc',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_template_id',
       'author_flair_text', 'author_flair_text_color', 'author_flair_type',
       'author_fullname', 'author_patreon_flair', 'author_premium', 'awarders',
       'body', 'can_gild', 'can_mod_post', 'collapsed',
       'collapsed_because_crowd_control', 'collapsed_reason', 'comment_type',
       'controversiality', 'created_utc', 'distinguished', 'edited', 'gilded',
       'gildings', 'id', 'is_submitter', 'link_id', 'locked', 'no_follow',
       'parent_id', 'permalink', 'quarantined', 'removal_reason',
       'retrieved_on', 'score', 'send_replies', 'stickied', 'subreddit',
       'subreddit_id', 'subreddit_name_prefixed', 'subreddit_type',
       'top_awarded_type', 'total_awards_received', 'treatment_tags',
       'arc

In [3]:
import pandas as pd
import networkx as nx

# load
comments_file = '1899_comments.zst'
submissions_file = '1899_submissions.zst'

# read data
df_comments = pd.read_json(comments_file, compression=dict(method='zstd', max_window_size=2147483648), lines=True)
df_submissions = pd.read_json(submissions_file, compression=dict(method='zstd', max_window_size=2147483648), lines=True)

# comment
df_comments['link_type'] = 'reply to a comment'
df_comments['link_id'] = df_comments['link_id'].apply(lambda x: x.split('_')[1] if pd.notnull(x) else x)
df_comments['parent_id'] = df_comments['parent_id'].apply(lambda x: x.split('_')[1] if pd.notnull(x) else x)
df_comments['child_id'] = df_comments['id']

# reply to a submission
df_comments.loc[df_comments['link_id'] == df_comments['parent_id'], 'link_type'] = 'reply to a submission'

# submission
df_submissions['child_id'] = df_submissions['id']
df_submissions['parent_id'] = 'Submission'
df_submissions['link_type'] = 'submission'

# merge
df_relations = pd.concat([
    df_comments[['child_id', 'parent_id', 'link_type']],
    df_submissions[['child_id', 'parent_id', 'link_type']]
], ignore_index=True)

# network
G = nx.from_pandas_edgelist(df_relations, 'parent_id', 'child_id', create_using=nx.DiGraph())

# caculation
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)

# df_relations
df_relations['degree'] = df_relations['child_id'].apply(lambda x: degree_centrality.get(x))
df_relations['betweenness'] = df_relations['child_id'].apply(lambda x: betweenness_centrality.get(x))
df_relations['closeness'] = df_relations['child_id'].apply(lambda x: closeness_centrality.get(x))


# save as csv
df_relations.to_csv('output_relations.csv', index=False)


In [ ]:
import pandas as pd
import networkx as nx

# load
comments_file = '1899_comments.zst'
submissions_file = '1899_submissions.zst'

# read data
df_comments = pd.read_json(comments_file, compression=dict(method='zstd', max_window_size=2147483648), lines=True)
df_submissions = pd.read_json(submissions_file, compression=dict(method='zstd', max_window_size=2147483648), lines=True)

# preprocessing comments
df_comments['link_id'] = df_comments['link_id'].str.split('_').str[-1]
df_comments['parent_id'] = df_comments['parent_id'].str.split('_').str[-1]
df_comments['child_id'] = df_comments['id']
df_comments['link_type'] = df_comments.apply(lambda x: 'reply to a submission' if x['link_id'] == x['parent_id'] else 'reply to a comment', axis=1)

# preprocess submissions
df_submissions['child_id'] = df_submissions['id']

# combine comments and submissions, but only comments have a 'parent_id'
df_relations = df_comments[['child_id', 'parent_id', 'link_type']].copy()

# build network
G = nx.from_pandas_edgelist(df_relations, 'parent_id', 'child_id', create_using=nx.DiGraph())

# calculate centrality measures
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)

# create a DataFrame for nodes with their centrality measures
nodes_df = pd.DataFrame(index=G.nodes())
nodes_df['degree'] = nodes_df.index.map(degree_centrality)
nodes_df['betweenness'] = nodes_df.index.map(betweenness_centrality)
nodes_df['closeness'] = nodes_df.index.map(closeness_centrality)

# save as csv
nodes_df.to_csv('output_nodes_centrality.csv', index=True, index_label='node_id')
